## 1. Configuration et Chargement

In [ ]:
# Imports
import sys
sys.path.append('..')

from config.spark_config import create_spark_session
from src.log_parser import LogParser
from src.sql_analytics import SQLAnalytics
from src.anomaly_detection import AnomalyDetector
from src.graph_analyzer import GraphAnalyzer

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

In [ ]:
# Créer la session Spark
spark = create_spark_session("LogAnalysisNotebook")
print(f"Spark Session créée: {spark.version}")

## 2. Parsing et Exploration

In [ ]:
# Parser les logs
DATA_FILE = '../data/NASA_access_log_full.txt'

parser = LogParser(spark)
logs_df = parser.parse_logs(DATA_FILE)

# Cache pour performance
logs_df.cache()

print(f"\nTotal de logs parsés: {logs_df.count():,}")

In [ ]:
# Explorer le schéma
logs_df.printSchema()

In [ ]:
# Aperçu des données
logs_df.show(10, truncate=False)

In [ ]:
# Statistiques descriptives
logs_df.describe(['status', 'bytes']).show()

## 3. Analyses SQL et KPI

In [ ]:
# Initialiser l'analyseur SQL
analytics = SQLAnalytics(spark)

In [ ]:
# Accès par heure
access_per_hour = analytics.access_count_per_hour(logs_df)

In [ ]:
# Top URLs
top_urls = analytics.top_urls(logs_df, top_n=20)

In [ ]:
# Top IPs
top_ips = analytics.top_ips(logs_df, top_n=20)

In [ ]:
# Distribution des codes HTTP
http_status = analytics.http_status_distribution(logs_df)

In [ ]:
# Analyse des erreurs
errors = analytics.error_analysis(logs_df)

In [ ]:
# Détection des pics
peaks = analytics.peak_activity_detection(logs_df)

### Visualisations des KPI

In [ ]:
# Graphique: Accès par heure
access_pd = access_per_hour.toPandas()

plt.figure(figsize=(15, 6))
plt.plot(access_pd.index, access_pd['access_count'], marker='o')
plt.title('Nombre d\'Accès par Heure', fontsize=16)
plt.xlabel('Heure')
plt.ylabel('Nombre d\'Accès')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Graphique: Top 10 URLs
top_urls_pd = top_urls.limit(10).toPandas()

plt.figure(figsize=(12, 6))
plt.barh(top_urls_pd['url'], top_urls_pd['hit_count'])
plt.title('Top 10 URLs les Plus Visitées', fontsize=16)
plt.xlabel('Nombre de Hits')
plt.ylabel('URL')
plt.tight_layout()
plt.show()

In [ ]:
# Graphique: Distribution des codes HTTP
status_pd = http_status.toPandas()

plt.figure(figsize=(10, 6))
plt.pie(status_pd.groupby('status_category')['count'].sum(), 
        labels=status_pd.groupby('status_category')['count'].sum().index,
        autopct='%1.1f%%',
        startangle=90)
plt.title('Distribution des Codes HTTP', fontsize=16)
plt.axis('equal')
plt.show()

## 4. Détection d'Anomalies avec MLlib

In [ ]:
# Initialiser le détecteur d'anomalies
detector = AnomalyDetector(spark)

In [ ]:
# K-Means Clustering
anomalies_kmeans = detector.detect_anomalies_kmeans(logs_df, k=5)

In [ ]:
# IPs suspectes
suspicious_ips = detector.get_suspicious_ips(anomalies_kmeans, top_n=50)

In [ ]:
# Détection statistique
anomalies_stats = detector.detect_statistical_anomalies(logs_df, std_threshold=3.0)

In [ ]:
# Analyse détaillée des comportements anormaux
anomalous_behavior = detector.analyze_anomalous_behavior(logs_df, anomalies_kmeans)

### Visualisation des Anomalies

In [ ]:
# Distribution des clusters
cluster_dist = anomalies_kmeans.groupBy('cluster').count().toPandas()

plt.figure(figsize=(10, 6))
plt.bar(cluster_dist['cluster'], cluster_dist['count'])
plt.title('Distribution des Clusters K-Means', fontsize=16)
plt.xlabel('Cluster')
plt.ylabel('Nombre d\'IPs')
plt.xticks(cluster_dist['cluster'])
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Top IPs suspectes
suspicious_pd = suspicious_ips.limit(15).toPandas()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Graphique 1: Nombre de requêtes
axes[0].barh(suspicious_pd['ip'], suspicious_pd['request_count'], color='coral')
axes[0].set_title('IPs Suspectes - Nombre de Requêtes', fontsize=14)
axes[0].set_xlabel('Requêtes')

# Graphique 2: Taux d'erreur
axes[1].barh(suspicious_pd['ip'], suspicious_pd['error_rate'] * 100, color='red')
axes[1].set_title('IPs Suspectes - Taux d\'Erreur', fontsize=14)
axes[1].set_xlabel('Taux d\'Erreur (%)')

plt.tight_layout()
plt.show()

## 5. Analyse de Graphe avec GraphX

In [ ]:
# Initialiser l'analyseur de graphe
graph_analyzer = GraphAnalyzer(spark)

In [ ]:
# Construire le graphe IP ↔ URL
graph = graph_analyzer.build_ip_url_graph(logs_df)

In [ ]:
# Analyser la connectivité des IPs
ip_connectivity = graph_analyzer.analyze_ip_connectivity(top_n=20)

In [ ]:
# Analyser la popularité des URLs
url_popularity = graph_analyzer.analyze_url_popularity(top_n=20)

In [ ]:
# Détecter les patterns suspects
suspicious_patterns = graph_analyzer.find_suspicious_patterns()

In [ ]:
# Exemple: Examiner les voisins d'une IP spécifique
# Remplacer par une IP de votre choix
sample_ip = top_ips.select('ip').first()['ip']
neighbors = graph_analyzer.get_ip_neighbors(sample_ip, limit=20)

## 6. Export des Résultats

In [ ]:
# Générer tous les KPI
kpis = analytics.generate_all_kpis(logs_df)

In [ ]:
# Sauvegarder en Parquet
analytics.save_kpis_to_parquet(kpis, '../output/parquet')

In [ ]:
# Export pour Grafana
from src.metrics_exporter import generate_grafana_metrics

kpis['suspicious_ips'] = suspicious_ips
kpis['ip_connectivity'] = ip_connectivity
kpis['url_popularity'] = url_popularity

generate_grafana_metrics(kpis, '../output/metrics')

In [ ]:
# Export du graphe
graph_analyzer.export_graph_for_visualization('../output')

## 7. Nettoyage

In [ ]:
# Unpersist les DataFrames
logs_df.unpersist()

# Arrêter Spark (optionnel)
# spark.stop()

---

## Analyse Terminée!

Les résultats sont disponibles dans `../output/`:
- Parquet files: `output/parquet/`
- Métriques Grafana: `output/metrics/`
- Graphe: `output/graph_vertices/` et `output/graph_edges/`

Pour visualiser dans Grafana, suivez les instructions dans `dashboard/GRAFANA_SETUP.md`